In [17]:
import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
import numpy as np
from PIL import Image
from pyspark.sql.functions import udf
from pyspark.sql.types import BinaryType
from pyspark.sql import SparkSession
import io

In [18]:
# Create the session
conf = SparkConf().\
                set('spark.ui.port', "4050").\
                set('spark.executor.memory', '4G').\
                set('spark.driver.memory', '45G').\
                set('spark.driver.maxResultSize', '10G').\
                setAppName("PySparkTutorial").\
                setMaster("local[*]")

# Create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=PySparkTutorial, master=local[*]) created by __init__ at C:\Users\leona\AppData\Local\Temp\ipykernel_9916\145051169.py:11 

In [19]:
spark

In [20]:
sc._conf.getAll()

[('spark.app.startTime', '1683375645046'),
 ('spark.app.name', 'PySparkTutorial'),
 ('spark.driver.memory', '45G'),
 ('spark.driver.host', 'rig.lan'),
 ('spark.sql.warehouse.dir',
  'file:/C:/Users/leona/PycharmProjects/BDC_project/spark-warehouse'),
 ('spark.executor.id', 'driver'),
 ('spark.driver.maxResultSize', '10G'),
 ('spark.app.id', 'local-1683375646662'),
 ('spark.ui.port', '4050'),
 ('spark.rdd.compress', 'True'),
 ('spark.driver.port', '54840'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.submit.pyFiles', ''),
 ('spark.submit.deployMode', 'client'),
 ('spark.executor.memory', '4G'),
 ('spark.ui.showConsoleProgress', 'true')]

In [49]:

df_appearances = spark.read.csv("archive/appearances.csv", header=True, inferSchema=True)
df_club_games = spark.read.csv("archive/club_games.csv", header=True, inferSchema=True)
df_clubs = spark.read.csv("archive/clubs.csv", header=True, inferSchema=True)
df_competitions = spark.read.csv("archive/competitions.csv", header=True, inferSchema=True)
df_game_events = spark.read.csv("archive/game_events.csv", header=True, inferSchema=True)
df_games = spark.read.csv("archive/games.csv", header=True, inferSchema=True)
df_player_valuations = spark.read.csv("archive/player_valuations.csv", header=True, inferSchema=True)
df_players = spark.read.csv("archive/players.csv", header=True, inferSchema=True)


In [23]:
#print number of rows for each dataset
print("Number of rows: ", df_appearances.count())
print("Number of rows: ", df_club_games.count())
print("Number of rows: ", df_clubs.count())
print("Number of rows: ", df_competitions.count())
print("Number of rows: ", df_game_events.count())
print("Number of rows: ", df_games.count())
print("Number of rows: ", df_player_valuations.count())
print("Number of rows: ", df_players.count())

df_appearances.printSchema()
df_game_events.printSchema()
df_games.printSchema()

df_appearances.show()
df_game_events.show()
df_games.show()


Number of rows:  1166215
Number of rows:  123096
Number of rows:  411
Number of rows:  43
Number of rows:  559853
Number of rows:  61548
Number of rows:  421564
Number of rows:  28503
root
 |-- appearance_id: string (nullable = true)
 |-- game_id: integer (nullable = true)
 |-- player_id: integer (nullable = true)
 |-- player_club_id: integer (nullable = true)
 |-- player_current_club_id: integer (nullable = true)
 |-- date: string (nullable = true)
 |-- player_name: string (nullable = true)
 |-- competition_id: string (nullable = true)
 |-- yellow_cards: integer (nullable = true)
 |-- red_cards: integer (nullable = true)
 |-- goals: integer (nullable = true)
 |-- assists: integer (nullable = true)
 |-- minutes_played: integer (nullable = true)

root
 |-- game_id: integer (nullable = true)
 |-- minute: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- club_id: integer (nullable = true)
 |-- player_id: integer (nullable = true)
 |-- description: string (nullable = true)

1 - unire players e appearances
2 - aggiungere la media punti per partita dei club per cui ha giocato in quell'anno
3 - aggiungere l'ultimo valore di mercato ottenuto in anno (player_valuations)

PRENDERE IN CONSIDERAZIONE LA POSSIBILITà DI ELIMINARE TOTALMENTE QUELLO CHE è SUCCESSO NEL 2023.

In [50]:
# join players and appearances
df_players_appearances = df_players.join(df_appearances, ["player_id"], how='inner')

In [51]:
df_players_appearances.show()

+---------+--------------------+---------------+-----------------+----------------------+-----------------+-------------+-------------+--------+--------------+-----+------------+-------------------+---------------------------+----------+------------------------+------------------------------------+----------+------------+--------------------+--------------------+-----------+--------------------+-------------+-------+--------------+----------------------+----------+--------------------+--------------+------------+---------+-----+-------+--------------+
|player_id|                name|current_club_id|current_club_name|country_of_citizenship| country_of_birth|city_of_birth|date_of_birth|position|  sub_position| foot|height_in_cm|market_value_in_eur|highest_market_value_in_eur|agent_name|contract_expiration_date|current_club_domestic_competition_id|first_name|   last_name|         player_code|           image_url|last_season|                 url|appearance_id|game_id|player_club_id|player_

In [52]:
#aggiungere a df_players_appearances la colonna year che contiene solo l'anno della feature date
df_players_appearances = df_players_appearances.withColumn("year", year(df_players_appearances.date))
#aggiungere a df_players_appearances la colonna month che contiene solo il mese della feature date
df_players_appearances = df_players_appearances.withColumn("month", month(df_players_appearances.date))
# we need also the day
df_players_appearances = df_players_appearances.withColumn("day", dayofmonth(df_players_appearances.date))

In [53]:
# drop da df_players_appearances le colonne current_club_id, current_club_name, city_of_birth, market_value_in_eur, contract_expiration_date, agent_name, current_club_domestic_competition_id, image_url, last_season, url, game_id, player_current_club_id
df_players_appearances = df_players_appearances.drop("current_club_id",  "appearance_id", "highest_market_value_in_eur", "current_club_name", "city_of_birth", "market_value_in_eur", "contract_expiration_date", "agent_name", "current_club_domestic_competition_id", "image_url", "last_season", "url", "game_id", "player_current_club_id", "first_name", "last_name", "player_name", "player_code")

In [96]:
df_players_appearances.filter(df_players_appearances.player_id == 148455).orderBy("year", "month").show(50)

+---------+-------------+----------------------+----------------+-------------+--------+------------+----+------------+--------------+----------+--------------+------------+---------+-----+-------+--------------+----+-----+----+
|player_id|         name|country_of_citizenship|country_of_birth|date_of_birth|position|sub_position|foot|height_in_cm|player_club_id|      date|competition_id|yellow_cards|red_cards|goals|assists|minutes_played|year|month| day|
+---------+-------------+----------------------+----------------+-------------+--------+------------+----+------------+--------------+----------+--------------+------------+---------+-----+-------+--------------+----+-----+----+
|   148455|Mohamed Salah|                 Egypt|           Egypt|   1992-06-15|  Attack|Right Winger|Left|         175|           631|      null|           CGB|           0|        0|    0|      0|            80|null| null|null|
|   148455|Mohamed Salah|                 Egypt|           Egypt|   1992-06-15|  Att

In [54]:
#aggiungere a df_players_appearances la colonna year che contiene solo l'anno della feature date
df_player_valuations = df_player_valuations.withColumn("year_v", year(df_player_valuations.date))
#aggiungere a df_players_appearances la colonna month che contiene solo il mese della feature date
df_player_valuations = df_player_valuations.withColumn("month_v", month(df_player_valuations.date))
#aggiungere a df_players_appearances la colonna day che contiene solo il giorno della feature date
df_player_valuations = df_player_valuations.withColumn("day_v", dayofmonth(df_player_valuations.date))

In [55]:
#dropping useless columns
df_player_valuations = df_player_valuations.drop("date", "datetime", "dateweek")

In [45]:
df_player_valuations.show()

+---------+---------------+-------------------+-----------------------------------+------+-------+-----+
|player_id|current_club_id|market_value_in_eur|player_club_domestic_competition_id|year_v|month_v|day_v|
+---------+---------------+-------------------+-----------------------------------+------+-------+-----+
|    99946|           1095|             150000|                                FR1|  2013|      8|    7|
|    99946|           1095|             100000|                                FR1|  2014|      1|   13|
|    76948|            979|             125000|                                PO1|  2010|      1|   29|
|    76948|            979|             250000|                                PO1|  2010|      8|   20|
|    76948|            979|             350000|                                PO1|  2011|      1|   17|
|    76948|            979|             375000|                                PO1|  2011|      8|   22|
|    76948|            979|             475000|        

In [99]:
# Join the two dataframes on player_id
df_joined = df_player_valuations.join(df_players_appearances, "player_id")

# Filter the rows based on the date conditions
df_filtered = df_joined.filter(
    (df_joined.year_v == df_joined.year+1) & (df_joined.month_v <= df_joined.month) |
    (df_joined.year_v == df_joined.year) & (df_joined.month_v > df_joined.month) |
    (df_joined.year_v == df_joined.year) & (df_joined.month_v == df_joined.month) & (df_joined.day_v > df_joined.day) |
    (df_joined.year_v == df_joined.year+1) & (df_joined.month_v == df_joined.month) & (df_joined.day_v <= df_joined.day)
)

# Group by the player_id and the valuation date and count the number of appearances
df_result2 = df_filtered.groupBy(
    "player_id", col("market_value_in_eur").alias("market_value"), "year_v", "month_v", "day_v", col("current_club_id").alias("club_id"), col("height_in_cm").alias("height"),  col("country_of_citizenship").alias("citizenship"), year("date_of_birth").alias("year_b"), "position", "sub_position")\
    .agg(collect_set("competition_id").alias("competition_id"),
        collect_set("player_club_id").alias("player_club_id"),
         count("date").alias("app"),
         sum("assists").alias("assists"),
         sum("goals").alias("goals"),
         sum("minutes_played").alias("minutes_played"),
         sum("red_cards").alias("red_cards"),
         sum("yellow_cards").alias("yellow_cards"))



In [107]:
df_joined.count()

27660316

In [105]:

df_filtered.count()

2671252

In [108]:
df_result2 = df_result2.withColumn("last_valuation", lag(df_result2.market_value).over(Window.partitionBy("player_id").orderBy("year_v", "month_v", "day_v")))

In [110]:
# Display the result
df_result2.show(50)

+---------+------------+------+-------+-----+-------+------+---------------+------+--------+------------------+--------------------+----------------+---+-------+-----+--------------+---------+------------+--------------+
|player_id|market_value|year_v|month_v|day_v|club_id|height|    citizenship|year_b|position|      sub_position|      competition_id|  player_club_id|app|assists|goals|minutes_played|red_cards|yellow_cards|last_valuation|
+---------+------------+------+-------+-----+-------+------+---------------+------+--------+------------------+--------------------+----------------+---+-------+-----+--------------+---------+------------+--------------+
|       65|     2500000|  2015|      1|   14|   1091|     0|       Bulgaria|  1981|  Attack|    Centre-Forward|           [CL, FR1]|           [162]| 18|      2|    5|          1336|        0|           4|          null|
|       65|     1000000|  2015|      6|    6|   1091|     0|       Bulgaria|  1981|  Attack|    Centre-Forward|     

In [73]:
df_players_appearances.filter((df_players_appearances.player_id == 148455) & (df_players_appearances.year == 2014)).show()

+---------+-------------+----------------------+----------------+-------------+--------+------------+----+------------+--------------+----------+--------------+------------+---------+-----+-------+--------------+----+-----+---+
|player_id|         name|country_of_citizenship|country_of_birth|date_of_birth|position|sub_position|foot|height_in_cm|player_club_id|      date|competition_id|yellow_cards|red_cards|goals|assists|minutes_played|year|month|day|
+---------+-------------+----------------------+----------------+-------------+--------+------------+----+------------+--------------+----------+--------------+------------+---------+-----+-------+--------------+----+-----+---+
|   148455|Mohamed Salah|                 Egypt|           Egypt|   1992-06-15|  Attack|Right Winger|Left|         175|           631|2014-09-30|            CL|           0|        0|    0|      0|             6|2014|    9| 30|
|   148455|Mohamed Salah|                 Egypt|           Egypt|   1992-06-15|  Attack|

In [47]:
#TABLE valutation: per ogni valutazione abbiamo la coppia (mese, anno) e vogliamo estrarre dalla TABLE presenze la somma delle presenze comprese tra (mese, anno-1 e mese,anno)
# month, year di appearances
# month_v, year_v di valutations
from pyspark.sql.functions import year, month

#inserire una colonna "prev_year" nella tabella player_appearances che contiene l'anno precedente a quello della colonna year
#df_players_appearances = df_players_appearances.withColumn("prev_year", df_players_appearances.year - 1)

# Unisci A con B utilizzando la chiave (player_id, mese_anno)
AB = df_players_appearances.join(df_player_valuations, ["player_id"], "inner")

AB.filter(AB.player_id == 148455).show()

+---------+-------------+----------------------+----------------+-------------+--------+------------+----+------------+--------------+--------------+------------+---------+-----+-------+--------------+----+-----+---+---------------+-------------------+-----------------------------------+------+-------+-----+
|player_id|         name|country_of_citizenship|country_of_birth|date_of_birth|position|sub_position|foot|height_in_cm|player_club_id|competition_id|yellow_cards|red_cards|goals|assists|minutes_played|year|month|day|current_club_id|market_value_in_eur|player_club_domestic_competition_id|year_v|month_v|day_v|
+---------+-------------+----------------------+----------------+-------------+--------+------------+----+------------+--------------+--------------+------------+---------+-----+-------+--------------+----+-----+---+---------------+-------------------+-----------------------------------+------+-------+-----+
|   148455|Mohamed Salah|                 Egypt|           Egypt|   19

In [31]:
#Filtriamo le istanze tenendo solo quelle che rispettano i vincoli sulle date
df_player_valutations_filtered = AB.filter(((AB["year"] == AB["year_v"]-1) & (AB["month"] >= AB["month_v"])) | ((AB["year"] == AB["year_v"]) & (AB["month"] <= AB["month_v"])))
#mostra
df_player_valutations_filtered.filter(df_player_valutations_filtered.player_id == 148455).show()

+---------+-------------+----------------------+----------------+-------------+--------+------------+----+------------+----------+-------------+-------------+--------------+-------------+--------------+------------+---------+-----+-------+--------------+----+-----+---+----------+----------+----------+---------------+-------------------+-----------------------------------+------+-------+
|player_id|         name|country_of_citizenship|country_of_birth|date_of_birth|position|sub_position|foot|height_in_cm|first_name|    last_name|  player_code|player_club_id|  player_name|competition_id|yellow_cards|red_cards|goals|assists|minutes_played|year|month|day|      date|  datetime|  dateweek|current_club_id|market_value_in_eur|player_club_domestic_competition_id|year_v|month_v|
+---------+-------------+----------------------+----------------+-------------+--------+------------+----+------------+----------+-------------+-------------+--------------+-------------+--------------+------------+-----

In [ ]:
################# RIPRENDERE DA QUI #######################

In [32]:
# drop da df_player_valutations_filtered le colonne date, datetime, current_club_id, player_club_domestic_competition_id, current_club_id, player_club_domestic_competition_id
df_player_valuations_filtered = df_player_valutations_filtered.drop("current_club_id", "player_club_domestic_competition_id")

In [33]:
df_player_valuations_filtered.filter(df_player_valutations_filtered.player_id == 148455).show()

+---------+-------------+----------------------+----------------+-------------+--------+------------+----+------------+-------------+--------------+-------------+--------------+------------+---------+-----+-------+--------------+----+-----+---+----------+-------------------+------+-------+
|player_id|         name|country_of_citizenship|country_of_birth|date_of_birth|position|sub_position|foot|height_in_cm|  player_code|player_club_id|  player_name|competition_id|yellow_cards|red_cards|goals|assists|minutes_played|year|month|day|  dateweek|market_value_in_eur|year_v|month_v|
+---------+-------------+----------------------+----------------+-------------+--------+------------+----+------------+-------------+--------------+-------------+--------------+------------+---------+-----+-------+--------------+----+-----+---+----------+-------------------+------+-------+
|   148455|Mohamed Salah|                 Egypt|           Egypt|   1992-06-15|  Attack|Right Winger|Left|         175|mohamed-

In [25]:
df_player_valutations_filtered.printSchema()

root
 |-- player_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- country_of_citizenship: string (nullable = true)
 |-- country_of_birth: string (nullable = true)
 |-- date_of_birth: date (nullable = true)
 |-- position: string (nullable = true)
 |-- sub_position: string (nullable = true)
 |-- foot: string (nullable = true)
 |-- height_in_cm: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- player_code: string (nullable = true)
 |-- player_club_id: integer (nullable = true)
 |-- player_name: string (nullable = true)
 |-- competition_id: string (nullable = true)
 |-- yellow_cards: integer (nullable = true)
 |-- red_cards: integer (nullable = true)
 |-- goals: integer (nullable = true)
 |-- assists: integer (nullable = true)
 |-- minutes_played: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- dateweek: date (nullable = true)
 |-- market_value_in

In [89]:
from pyspark.sql.functions import sum, col

df_new = df_player_valutations_filtered.groupBy("player_id", "competition_id", "player_club_id", "year_v", "month_v", "market_value_in_eur").agg(
    sum(col("yellow_cards")).alias("yellow_cards"),
    sum(col("red_cards")).alias("red_cards"),
    sum(col("goals")).alias("goals"),
    sum(col("assists")).alias("assists"),
    sum(col("minutes_played")).alias("minutes_played")
).select("player_id", "competition_id", "player_club_id", "month_v", "year_v", "yellow_cards", "red_cards", "goals", "assists", "minutes_played", "market_value_in_eur").distinct()

In [91]:
#df_player_valutations_filtered.filter(df_player_valutations_filtered.player_id == 148455).show()
df_new.filter(df_new.player_id == 148455).orderBy(df_new.year_v, df_new.month_v).show(10)

+---------+--------------+--------------+-------+------+------------+---------+-----+-------+--------------+-------------------+
|player_id|competition_id|player_club_id|month_v|year_v|yellow_cards|red_cards|goals|assists|minutes_played|market_value_in_eur|
+---------+--------------+--------------+-------+------+------------+---------+-----+-------+--------------+-------------------+
|   148455|           IT1|           430|      2|  2015|           0|        0|    2|      1|           131|           10000000|
|   148455|           GB1|           631|      2|  2015|           0|        0|    0|      0|            30|           10000000|
|   148455|           FAC|           631|      2|  2015|           0|        0|    0|      1|            70|           10000000|
|   148455|            CL|           631|      2|  2015|           0|        0|    0|      0|            77|           10000000|
|   148455|            EL|           430|      2|  2015|           1|        0|    1|      0|    

In [35]:
#TEST
print(df_new.filter(df_new.player_id == 148455).count())
df_new.filter(df_new.player_id == 148455).agg(sum(col("goals"))).show()

114
+----------+
|sum(goals)|
+----------+
|       574|
+----------+



In [92]:
from pyspark.sql.functions import collect_set, sum

# Raggruppare per (player_id, year) e raccogliere i valori unici di competition_id e player_club_id in liste
df_grouped = df_new.groupBy("player_id", "year_v", "month_v", "market_value_in_eur") \
              .agg(collect_set("competition_id").alias("competition_id_list"),
                   collect_set("player_club_id").alias("player_club_id_list"))

# Unire il dataframe originale con quello ottenuto al passaggio precedente sulla chiave (player_id, year)
# e calcolare la somma delle colonne yellow_cards, red_cards, goals, assists e minutes_played raggruppando sulla nuova chiave (player_id, competition_id, year, player_club_id)
df_result = df_new.join(df_grouped, ["player_id", "year_v", "month_v", "market_value_in_eur"]) \
              .groupBy("player_id", "competition_id", "month_v", "year_v", "player_club_id", "market_value_in_eur") \
              .agg(sum("yellow_cards").alias("yellow_cards"),
                   sum("red_cards").alias("red_cards"),
                   sum("goals").alias("goals"),
                   sum("assists").alias("assists"),
                   sum("minutes_played").alias("minutes_played")) \
              .groupBy("player_id", "year_v", "month_v", "market_value_in_eur") \
              .agg(collect_set("competition_id").alias("competition_id"),
                   collect_set("player_club_id").alias("player_club_id"),
                   sum("yellow_cards").alias("yellow_cards"),
                   sum("red_cards").alias("red_cards"),
                   sum("goals").alias("goals"),
                   sum("assists").alias("assists"),
                   sum("minutes_played").alias("minutes_played"))

# Visualizza il risultato
df_result.show()

+---------+------+-------+-------------------+--------------------+--------------+------------+---------+-----+-------+--------------+
|player_id|year_v|month_v|market_value_in_eur|      competition_id|player_club_id|yellow_cards|red_cards|goals|assists|minutes_played|
+---------+------+-------+-------------------+--------------------+--------------+------------+---------+-----+-------+--------------+
|       10|  2015|      1|            1000000|          [CIT, IT1]|         [398]|           2|        0|    6|      5|           819|
|       65|  2015|      1|            2500000|           [CL, FR1]|         [162]|           4|        0|    6|      2|          1508|
|       65|  2015|      6|            1000000|           [CL, FR1]|         [162]|           4|        0|    8|      2|          2274|
|       65|  2015|      9|            1000000|  [CL, FR1, EL, GR1]|   [1091, 162]|           2|        0|    7|      1|          2108|
|      532|  2018|      6|             500000|         

In [93]:
df_result.filter(df_result.player_id == 148455).orderBy(df_result.year_v, df_result.month_v).show()

+---------+------+-------+-------------------+--------------------+--------------+------------+---------+-----+-------+--------------+
|player_id|year_v|month_v|market_value_in_eur|      competition_id|player_club_id|yellow_cards|red_cards|goals|assists|minutes_played|
+---------+------+-------+-------------------+--------------------+--------------+------------+---------+-----+-------+--------------+
|   148455|  2015|      2|           10000000|[CL, FAC, EL, IT1...|    [430, 631]|           1|        0|    3|      2|           488|
|   148455|  2015|      3|           12000000|[CL, FAC, CIT, EL...|    [430, 631]|           1|        0|    6|      3|           900|
|   148455|  2015|      7|           18000000|[CL, FAC, CIT, EL...|    [430, 631]|           1|        0|    9|      5|          1970|
|   148455|  2015|     11|           20000000|[CL, FAC, CIT, EL...|[12, 430, 631]|           3|        0|   15|      5|          3113|
|   148455|  2016|      1|           20000000|[CL, FAC,

In [39]:
#df_new.filter(df_new.player_id == 148455).show(100)
df_result.filter(df_result.player_id == 148455).show(100)

+---------+------+-------+-------------------+--------------------+--------------+------------+---------+-----+-------+--------------+
|player_id|year_v|month_v|market_value_in_eur|      competition_id|player_club_id|yellow_cards|red_cards|goals|assists|minutes_played|
+---------+------+-------+-------------------+--------------------+--------------+------------+---------+-----+-------+--------------+
|   148455|  2015|      2|           10000000|[CL, FAC, EL, IT1...|    [430, 631]|           1|        0|    3|      2|           488|
|   148455|  2015|      3|           12000000|[CL, FAC, CIT, EL...|    [430, 631]|           1|        0|    6|      3|           900|
|   148455|  2015|      7|           18000000|[CL, FAC, CIT, EL...|    [430, 631]|           1|        0|    9|      5|          1970|
|   148455|  2015|     11|           20000000|[CL, FAC, CIT, EL...|[12, 430, 631]|           3|        0|   15|      5|          3113|
|   148455|  2016|      1|           20000000|[CL, FAC,

In [104]:
df_result.count()

141441

In [40]:
#TEST (a questo punto il test non ha piu valore dal momento che si tiene conto per ogni valutazione quello che si è fatto l'anno precedente a year_v--month_v)
print(df_result.filter(df_result.player_id == 148455).count())
df_result.filter(df_result.player_id == 148455).agg(sum(col("goals"))).show()

23
+----------+
|sum(goals)|
+----------+
|       574|
+----------+



In [43]:
#unire df_result con df_players ma di df_players non voglio la colonna "market_value_in_eur"
df_players = df_players.drop("market_value_in_eur")
df_stats = df_result.join(df_players, ["player_id"], "inner")
df_stats.filter(df_stats.player_id == 148455).show(100)

+---------+------+-------+-------------------+--------------------+--------------+------------+---------+-----+-------+--------------+-------------+---------------+-----------------+----------------------+----------------+-----------------+-------------+--------+------------+----+------------+---------------------------+----------+------------------------+------------------------------------+----------+-------------+-------------+--------------------+-----------+--------------------+
|player_id|year_v|month_v|market_value_in_eur|      competition_id|player_club_id|yellow_cards|red_cards|goals|assists|minutes_played|         name|current_club_id|current_club_name|country_of_citizenship|country_of_birth|    city_of_birth|date_of_birth|position|sub_position|foot|height_in_cm|highest_market_value_in_eur|agent_name|contract_expiration_date|current_club_domestic_competition_id|first_name|    last_name|  player_code|           image_url|last_season|                 url|
+---------+------+----

In [44]:
df_stats = df_stats.drop("current_club_id", "current_club_name", "city_of_birth", "contract_expiration_date", "agent_name", "current_club_domestic_competition_id", "image_url", "last_season", "url", "player_current_club_id")

In [47]:
df_stats = df_stats.drop("highest_market_value", "player_code")

In [48]:
df_stats.filter(df_stats.player_id == 148455).show(100)

+---------+------+-------+-------------------+--------------------+--------------+------------+---------+-----+-------+--------------+-------------+----------------------+----------------+-------------+--------+------------+----+------------+---------------------------+----------+-------------+
|player_id|year_v|month_v|market_value_in_eur|      competition_id|player_club_id|yellow_cards|red_cards|goals|assists|minutes_played|         name|country_of_citizenship|country_of_birth|date_of_birth|position|sub_position|foot|height_in_cm|highest_market_value_in_eur|first_name|    last_name|
+---------+------+-------+-------------------+--------------------+--------------+------------+---------+-----+-------+--------------+-------------+----------------------+----------------+-------------+--------+------------+----+------------+---------------------------+----------+-------------+
|   148455|  2015|      2|           10000000|[CL, FAC, EL, IT1...|    [430, 631]|           1|        0|    3| 

In [49]:
print("Number of rows: ", df_stats.count())

Number of rows:  141441


In [1]:
# a df_stats aggiungere la colonna età (semplicemnte data di nascita - (year_v-month_v) ---> putroppo abbiamo perso il giorno della data della valutazione, per recuperarla si deve evitare di
# perdere o scartare la colonna dateweek che è ancora presente nella tabella data in output nella cella 21 of 34 df_player_valutations_filtered)

# a df_stats aggiungere la colonna partite_giocate che contiene il numero di partite giocate dalle sue squadre (player_club_id) in year;

# a df_stats aggiungere la colonna punti_ottenuti che contiene il numero di punti ottenuti dalle sue squadre (player_club_id) in year; (3 punti per vittoria, 1 per pareggio, 0 per sconfitta)

# a df_stats aggiungere la colonna media_punti che contiene la media dei punti ottenuti dalle sue squadre (player_club_id) in year; (ottenerla dalle colonne partite_giocate e punti_ottenuti)